In [1]:
import sys
sys.path.append('../')
import conproc

In [2]:
import os

class EchoProcess(conproc.BaseWorkerProcess):
    '''Process that sends back the same data it receives.'''
    
    def __call__(self):
        '''Main process loop.'''
        print(f'Starting process {os.getpid()}')
        while True:
            data = self.messenger.receive_data(blocking=True)
            print(f'process {os.getpid()} received: {data}')
            self.messenger.send_reply(data)

resource = conproc.WorkerResource(EchoProcess)
resource

WorkerResource(worker_process_type=<class '__main__.EchoProcess'>, method=None, _proc=None, _messenger=None)

In [3]:
with conproc.WorkerResource(EchoProcess) as w:
    w.messenger.send_data_noreply('Hello, world!')
    
    print('received:', w.messenger.receive_data())

Starting process 766121


process 766121 received: Hello, world!
received: Hello, world!


In [4]:
with conproc.WorkerResource(EchoProcess) as w:
    w.messenger.send_request('Hello, world!')
    
    print(f'{w.messenger.remaining()=}')
    
    print('received:', w.messenger.receive_data())
    
    print(f'{w.messenger.remaining()=}')

Starting process 766131


process 766131 received: Hello, world!
w.messenger.remaining()=1
received: Hello, world!
w.messenger.remaining()=0


In [8]:
with conproc.WorkerResource(EchoProcess) as w:
    w.messenger.send_request('Hello, world!')
    w.messenger.send_request('Hello, world!')
    w.messenger.send_request('Hello, world!')
    
    print(f'{w.messenger.remaining()=}')
    
    while w.messenger.remaining() > 0:
        print('received:', w.messenger.receive_data())

Starting process 766245


process 766245 received: Hello, world!
process 766245 received: Hello, world!
process 766245 received: Hello, world!
w.messenger.remaining()=3
received: Hello, world!
received: Hello, world!
received: Hello, world!


In [5]:
resource.start()
print(f'{resource.is_alive()=}')

resource.messenger.send_data_noreply('hello')
print(f'{resource.is_alive()=}')

#resource.messenger.send_close_request()
#print(f'{resource.is_alive()=}')

import time
time.sleep(0.1)

resource.join()

Starting process 766140
resource.is_alive()=True
resource.is_alive()=True
process 766140 received: hello


Process ForkProcess-3:
Traceback (most recent call last):
  File "/home/devin/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/devin/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_766078/439355911.py", line 10, in __call__
    data = self.messenger.receive_data(blocking=True)
  File "/DataDrive/projects/concurrent/examples/../conproc/messenger/prioritymessenger.py", line 85, in receive_data
    return self._receive_message(blocking=blocking).payload
  File "/DataDrive/projects/concurrent/examples/../conproc/messenger/prioritymessenger.py", line 103, in _receive_message
    raise ResourceRequestedClose(f'Resource requested that this process close.')
conproc.messenger.exceptions.ResourceRequestedClose: Resource requested that this process close.


In [6]:
resource.start()

# send a message and wait for each reply
for i in range(3):
    resource.messenger.request_data(f'Hello, world {i}!')
    print(resource.messenger.receive_data())

resource.terminate()

Starting process 766165


AttributeError: 'PriorityMessenger' object has no attribute 'request_data'

In [ ]:
resource.start()

# send a bunch of messages
for i in range(3):
    resource.messenger.request_data(f'Hello, world {i}!')

print(f'{resource.messenger.remaining()} are now in the receive queue')

# receive all of the messages
while resource.messenger.remaining():
    print(resource.messenger.receive_data())
    print(resource.messenger.remaining(), 'remain')

resource.terminate()

3 are now in the receive queue
Hello, world 0!
2 remain
Hello, world 1!
1 remain
Hello, world 2!
0 remain


In [ ]:
class EchoProcess:
    '''Simply sends back the same data it receives.'''
    
    def __init__(self, messenger: conproc.PriorityMessenger):
        '''Accept data that should be passed to the worker prior to starting.'''
        self.messenger = messenger
    
    def __call__(self):
        '''Main process loop.'''
        while True:
            data = self.messenger.receive_data(blocking=True)
            print(f'process {os.getpid()} received: {data}')
            self.messenger.send_reply(data)


Process ForkProcess-2:
Traceback (most recent call last):
  File "/home/devin/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/devin/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_761161/2983712441.py", line 11, in __call__
    data = self.messenger.receive_data(blocking=True)
  File "/DataDrive/projects/concurrent/examples/../conproc/messenger/prioritymessenger.py", line 85, in receive_data
    return self._receive_message(blocking=blocking).payload


In [ ]:
wr.is_alive()

True

In [ ]:
wr.messenger.available()

False